[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/yejingxin/ai-on-gke/blob/ipp/applications/ipyparallel/tpu/example_notebook/quickstart.ipynb)
[![Open On GitHub](https://img.shields.io/badge/Open-on%20GitHub-blue?logo=GitHub)](https://github.com/yejingxin/ai-on-gke/blob/ipp/applications/ipyparallel/tpu/example_notebook/quickstart.ipynb)

# Quick Start Guide: Running Notebooks on Multi-host TPU
This tutorial will guide you through initializing a notebook setup and running example cells on a multi-host TPU. This guide focuses on executing cells on an existing service. For information on setting up the service, please refer to this [user guide](https://github.com/yejingxin/ai-on-gke/blob/ipp/applications/ipyparallel/README.md).

## Install `ipyparallel`
To interact with the cluster, we use IPython Parallel and cell magic. You can either have it pre-installed in the jupyter-notebook container image or install it using the following cell:

In [1]:
!pip install ipyparallel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.1/293.1 kB 7.1 MB/s eta 0:00:0000:01


## Connect to the TPU cluster
When the multi-host notebook service starts, it generates a client config file in the filestore folder. Use this file to connect to different hosts in the TPU Cluster:

In [1]:
import ipyparallel as ipp
IPP_FILE_PATH = "nfs/security/ipcontroller-client.json"
rc = ipp.Client(IPP_FILE_PATH)
print(rc.ids)
if rc.ids:
 print(f'Successfully established connection with {len(rc.ids)} hosts')
else:
 print(f'Failed to connect to {IPP_FILE_PATH}')

[2, 3]
Successfully established connection with 2 hosts


##  Check Current Task Status
A ready-to-use TPU cluster should have zero outstanding tasks, and all task queues should be marked as completed:

In [4]:
print('Cluster task queue status:', rc.queue_status())
print('Current outstanding tasks:', rc.outstanding)

Cluster task queue status: {'unassigned': 0, 0: {'queue': 0, 'completed': 2, 'tasks': 0}, 1: {'queue': 0, 'completed': 2, 'tasks': 0}, 2: {'queue': 0, 'completed': 2, 'tasks': 0}, 3: {'queue': 0, 'completed': 2, 'tasks': 0}}
Current outstanding tasks: set()


## Run a cell on different hosts
We use the `%%px --block --group-outputs=engine` cell magic to execute code across hosts in blocking mode. For detailed instructions on cell magic, refer to the `ipyparallel` [documentation](https://ipyparallel.readthedocs.io/en/latest/tutorial/magics.html).  

Example:

In [5]:
%%px --block --group-outputs=engine
import jax
import socket
print(f'jax process {jax.process_index():02d} is running on {socket.gethostname()}, \
local num of chips: {jax.local_device_count()}, global num of chips: {jax.device_count()}')

[stdout:2] jax process 01 is running on ipp-notebook-0-2, local num of chips: 4, global num of chips: 8


[stdout:3] jax process 00 is running on ipp-notebook-0-1, local num of chips: 4, global num of chips: 8


This output demonstrates different JAX processes running on various hosts and displays the total number of chips within the TPU cluster in synchronous mode.

## Clean Up
Ensure all the tasks are completed and disconnect from the cluster.

In [19]:
print('Cluster task queue status:', rc.queue_status())
print('Current outstanding tasks:', rc.outstanding)

Cluster task queue status: {'unassigned': 0, 0: {'queue': 0, 'completed': 2, 'tasks': 0}, 1: {'queue': 0, 'completed': 2, 'tasks': 0}, 2: {'queue': 0, 'completed': 2, 'tasks': 0}, 3: {'queue': 0, 'completed': 2, 'tasks': 0}}
Current outstanding tasks: set()


In [22]:
rc.purge_everything()

In [5]:
rc.shutdown()

**Important**: The step above only disconnect the notebook from the cluster. The notebook service and cluster itself remain active. For complete resource cleanup, including shutting down the service and deleting cluster resources, please refer to the "Clean Up" section in the cluster [user guide](../../README.md). Proper cleanup ensures efficient resource management and prevents unnecessary costs.